In [9]:
# Step 1: Get data
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import cv2
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import pandas as pd
import numpy as np
import os

# Simple CNN model for feature extraction
class FeatureExtractorCNN(nn.Module):
    def __init__(self):
        super(FeatureExtractorCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.fc1 = nn.Linear(32 * 64 * 64, 128)  # Assuming input size is (3, 256, 256)
        self.fc2 = nn.Linear(128, 64)  # Reduce to 64 features, no risk classification
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 32 * 64 * 64)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

google_drive_path = "/content/drive/MyDrive/Colab Notebooks/Human Signals-Final Project/data/"

# Load videos
thermal_video_path = google_drive_path + 'aerial_data' + '/' + 'danger_running-thermal.MP4'
normal_video_path = google_drive_path + 'aerial_data' + '/' + 'danger_running-regular.MP4'

thermal_cap = cv2.VideoCapture(thermal_video_path)
normal_cap = cv2.VideoCapture(normal_video_path)

thermal_fps = thermal_cap.get(cv2.CAP_PROP_FPS)
normal_fps = normal_cap.get(cv2.CAP_PROP_FPS)

print(f"Thermal video FPS: {thermal_fps}")
print(f"Normal video FPS: {normal_fps}")

# Initialize model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = FeatureExtractorCNN().to(device)
model.eval()  # We assume the model is pre-trained, here for simplicity we just use random weights

transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])

results = []

frame_idx = 0
while True:
    ret_thermal, frame_thermal = thermal_cap.read()
    ret_normal, frame_normal = normal_cap.read()

    if not ret_thermal or not ret_normal:
        break

    # Extract average temperature (assuming brightness = temperature proxy)
    gray_thermal = cv2.cvtColor(frame_thermal, cv2.COLOR_BGR2GRAY)
    avg_temperature = np.mean(gray_thermal)

    # Estimate number of people using YOLO (if using YOLO for counting people)
    # Assuming YOLO is set up as previously mentioned for people counting
    gray_normal = cv2.cvtColor(frame_normal, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray_normal, 200, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    avg_people_count = len(contours)

    # Extract features using CNN
    input_frame = transform(frame_normal).unsqueeze(0).to(device)
    with torch.no_grad():
        features = model(input_frame)
        features = features.cpu().numpy().flatten()  # Flatten the feature vector to store

    seconds_elapsed = frame_idx / normal_fps  # Assume thermal and normal videos are synchronized

    results.append([seconds_elapsed, avg_temperature, avg_people_count] + list(features))
    frame_idx += 1

# Save results to CSV
columns = ["seconds_elapsed", "average_temperature", "average_people_count"] + [f"feature_{i+1}" for i in range(64)]
output_df = pd.DataFrame(results, columns=columns)
output_df.to_csv(google_drive_path+'danger_running_video_features.csv', index=False)

thermal_cap.release()
normal_cap.release()

print("Feature extraction completed and saved to video_features.csv")


Thermal video FPS: 29.97002997002997
Normal video FPS: 29.97002997002997
Feature extraction completed and saved to video_features.csv


In [10]:
import cv2
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import pandas as pd
import numpy as np
import os

# Simple CNN model for feature extraction
class FeatureExtractorCNN(nn.Module):
    def __init__(self):
        super(FeatureExtractorCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.fc1 = nn.Linear(32 * 64 * 64, 128)  # Assuming input size is (3, 256, 256)
        self.fc2 = nn.Linear(128, 64)  # Reduce to 64 features, no risk classification
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 32 * 64 * 64)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x


def extract_aerial_features(label):
  activity_name , location = label.split("-", 1)
  google_drive_path = "/content/drive/MyDrive/Colab Notebooks/Human Signals-Final Project/data/"

  # Load videos
  thermal_video_path = google_drive_path + 'aerial_data' + '/' + activity_name + "-thermal.MP4"
  normal_video_path = google_drive_path + 'aerial_data' + '/' + activity_name + "-regular.MP4"

  thermal_cap = cv2.VideoCapture(thermal_video_path)
  normal_cap = cv2.VideoCapture(normal_video_path)

  thermal_fps = thermal_cap.get(cv2.CAP_PROP_FPS)
  normal_fps = normal_cap.get(cv2.CAP_PROP_FPS)

  print(f"Thermal video FPS: {thermal_fps}")
  print(f"Normal video FPS: {normal_fps}")

  # Initialize model
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  model = FeatureExtractorCNN().to(device)
  model.eval()  # We assume the model is pre-trained, here for simplicity we just use random weights

  transform = transforms.Compose([
      transforms.ToPILImage(),
      transforms.Resize((256, 256)),
      transforms.ToTensor()
  ])

  results = []

  frame_idx = 0
  while True:
      ret_thermal, frame_thermal = thermal_cap.read()
      ret_normal, frame_normal = normal_cap.read()

      if not ret_thermal or not ret_normal:
          break

      # Extract average temperature (assuming brightness = temperature proxy)
      gray_thermal = cv2.cvtColor(frame_thermal, cv2.COLOR_BGR2GRAY)
      avg_temperature = np.mean(gray_thermal)

      # Estimate number of people using YOLO (if using YOLO for counting people)
      # Assuming YOLO is set up as previously mentioned for people counting
      gray_normal = cv2.cvtColor(frame_normal, cv2.COLOR_BGR2GRAY)
      _, thresh = cv2.threshold(gray_normal, 200, 255, cv2.THRESH_BINARY)
      contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
      avg_people_count = len(contours)

      # Extract features using CNN
      input_frame = transform(frame_normal).unsqueeze(0).to(device)
      with torch.no_grad():
          features = model(input_frame)
          features = features.cpu().numpy().flatten()  # Flatten the feature vector to store

      seconds_elapsed = frame_idx / normal_fps  # Assume thermal and normal videos are synchronized

      results.append([seconds_elapsed, avg_temperature, avg_people_count] + list(features))
      frame_idx += 1

  # Save results to CSV
  columns = ["seconds_elapsed", "average_temperature", "average_people_count"] + [f"feature_{i+1}" for i in range(64)]
  output_df = pd.DataFrame(results, columns=columns)
  output_df['activity'] = activity_name


  thermal_cap.release()
  normal_cap.release()

  return output_df



In [11]:
labels = ["danger_running-in_school","play_basketball_alone-in_school","play_basketball_with_kid-in_school","running-in_school","walking-in_school","walking_at_7_foor-in_school","walking_while_using_iphone-in_school"]
all_aerial_features_list = []
for label in labels:
  aerial_features_df = extract_aerial_features(label)
  all_aerial_features_list.append(aerial_features_df)

# Concatenate all DataFrames into one
all_aerial_features_df = pd.concat(all_aerial_features_list, ignore_index=True)

google_drive_path = "/content/drive/MyDrive/Colab Notebooks/Human Signals-Final Project/data/"
all_aerial_features_df.to_csv(google_drive_path+'all_aerial_features.csv', index=False)
print("File saved as 'all_aerial_features.csv")

Thermal video FPS: 29.97002997002997
Normal video FPS: 29.97002997002997
Thermal video FPS: 29.95891349007076
Normal video FPS: 29.97002997002997
Thermal video FPS: 29.967156249637792
Normal video FPS: 29.97002997002997
Thermal video FPS: 29.97002997002997
Normal video FPS: 29.97002997002997
Thermal video FPS: 29.97002997002997
Normal video FPS: 29.97002997002997
Thermal video FPS: 29.97002997002997
Normal video FPS: 29.962480844092934
Thermal video FPS: 29.97002997002997
Normal video FPS: 29.97002997002997
File saved as 'all_aerial_features.csv
